## Question

The median is the middle value in an ordered integer list. If the size of the list is even, there is no middle value and the median is the mean of the two middle values.

    For example, for arr = [2,3,4], the median is 3.
    For example, for arr = [2,3], the median is (2 + 3) / 2 = 2.5.

Implement the MedianFinder class:

    MedianFinder() initializes the MedianFinder object.
    void addNum(int num) adds the integer num from the data stream to the data structure.
    double findMedian() returns the median of all elements so far. Answers within 10-5 of the actual answer will be accepted.


## Solution

There are several ways to implement this. The most naive way to think of this problem is to sort the array over and over again when `addNum` is called. However, there is better data structure called heap which we can use to maintain the maximum number on the left half of the array (using max heap) and the minimum number on the right half of the array (using min heap). Inserting elements to a heap can be done in logarithmic time.

When `findMedian` is called, we just need to check whether the heaps are of same size (which means the original data array has even number of elements; return the average of two middle elements) or of different size (which means the original data array has odd number of elements; return the middle element).

If heaps are of same size, we fetch the maximum element from the max heap and minimum element from the min heap and return their average.
If heaps are of different size, we fetch the minimum element from the min heap.

Python's `heapq` implementation is min heap so in order to make it behave like a max heap, we will have to covert the positive numbers to negative numbers when doing `heapq.heappush`

In [1]:
import heapq


class MedianFinder:
    def __init__(self):
        self.left_half = []  # max heap
        self.right_half = []  # min heap

    def addNum(self, num: int) -> None:
        if len(self.left_half) == len(self.right_half):
            # Push to left half first (want to behave this like max heap so have to flip sign),
            # Pop the maximum element from left half (re-flip the sign before pushing to right half so that we get the original number)
            # Push that element to right half
            # heapq.heappushpop is actually two operations i) push to heap first and then 2) pop the root element from the heap
            heapq.heappush(self.right_half, -heapq.heappushpop(self.left_half, -num))
            # After these three operations, right half will have +1 element comapred to left half
        else:
            # If the heaps are of different sizes (i.e. right half has +1 element compared to left half), do the opposite
            # During the push of heappushpop, right half will have +2 elements compared to left half
            # But the root element of right half heap will be popped during pop operation
            # This popped element will be pushed to left half
            heapq.heappush(self.left_half, -heapq.heappushpop(self.right_half, num))
            # After these three operations, both halves will have equal size

    def findMedian(self) -> float:
        if len(self.left_half) == len(self.right_half):
            # Since we flipped the signs of numbers when pushing to left_half, we have to flip it back
            # The root element of max heap is maximum number and the root element of min heap is minimum number
            # With array representation, the root elements are 0-th index of the array
            return (-self.left_half[0] + self.right_half[0]) / 2
        else:
            return self.right_half[0]

    def __str__(self) -> str:
        left_half_str = f"Left Half: {[-x for x in self.left_half]}"
        right_half_str = f"Right Half: {self.right_half}"
        return left_half_str + " " + right_half_str

Runtime for `addNum`: O(log(n)); there are three heap operations; push, pop and push for each `addNum` call. Each of these heap operations take O(log(n)) time so the total time taken will be 3 * O(log(n)) -> O(log(n))

Runtime for `findMedian`: O(1)

Space: O(n)

In [2]:
from collections import deque

# test cases for the problem
# operations: 'a' -> addNum, 'f' -> findMedian

test_cases = [
    {
        "data_stream": [5, 6, 8, 3, 4, 6, 8, 3, 2],
        "operations": deque(["a", "a", "a", "f", "a", "a", "f"]),
        "answers": deque([None, None, None, 6, None, None, 5]),
    },
    {
        "data_stream": [-2, -6, 8, 33, 44, 0, 0, 0, -2],
        "operations": deque(["a", "a", "a", "f", "a", "a", "f"]),
        "answers": deque([None, None, None, -2, None, None, 8]),
    },
    {
        "data_stream": [0, 0, 0, 0, 0, 0, 0, 0],
        "operations": deque(["a", "a", "a", "f", "a", "a", "f"]),
        "answers": deque([None, None, None, 0, None, None, 0]),
    },
]

In [3]:
for test_case in test_cases:

    test_median_finder = MedianFinder()
    counter = 0

    while test_case["operations"]:

        num = test_case["data_stream"][counter]
        op = test_case["operations"].popleft()
        answer = test_case["answers"].popleft()

        if op == "a":
            test_median_finder.addNum(num)
            print(test_median_finder)
            counter += 1
        elif op == "f":
            result = test_median_finder.findMedian()
            assert result == answer, f"Expected {answer}; Got back {result}"
    print("Passed")

Left Half: [] Right Half: [5]
Left Half: [5] Right Half: [6]
Left Half: [5] Right Half: [6, 8]
Left Half: [5, 3] Right Half: [6, 8]
Left Half: [4, 3] Right Half: [5, 8, 6]
Passed
Left Half: [] Right Half: [-2]
Left Half: [-6] Right Half: [-2]
Left Half: [-6] Right Half: [-2, 8]
Left Half: [-2, -6] Right Half: [8, 33]
Left Half: [-2, -6] Right Half: [8, 33, 44]
Passed
Left Half: [] Right Half: [0]
Left Half: [0] Right Half: [0]
Left Half: [0] Right Half: [0, 0]
Left Half: [0, 0] Right Half: [0, 0]
Left Half: [0, 0] Right Half: [0, 0, 0]
Passed
